## app backend settings and internal functions

In [ ]:
from IPython import get_ipython

In [ ]:
import pandas as pd
import hvplot.pandas  # Adds .hvplot and .interactive methods to Pandas dataframes
import panel as pn  # Panel is a simple, flexible and enterprise-ready data app framework

pn.extension(loading_spinner='dots', loading_color='#4d8060', sizing_mode="stretch_width", template="fast")
pd.set_option("display.precision", 0)

PALETTE = [
    "#ff6f69",
    "#ffcc5c",
    "#88d8b0",
]
ACCENT_BASE_COLOR = PALETTE[2]

In [ ]:
import holoviews as hv
from holoviews import opts
import matplotlib.pyplot as plt
from typing import List, Set, Dict, Tuple, Union, Any
from dataclasses import dataclass
import datashader as ds
import colorcet as cc
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize

import holoviews.plotting.mpl
import holoviews.plotting.bokeh

from pyteomics import mzml, auxiliary
import numba
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import numpy  as np
from scipy.optimize import curve_fit
from collections import defaultdict

from alphapept.constants import averagine_aa, averagine_avg
from alphapept.constants import isotopes as averagine_iso
from alphapept.chem import get_average_formula, mass_to_dist

from bokeh.palettes import Magma, Inferno, Plasma, Viridis, Cividis
from holoviews.plotting.util import bokeh_palette_to_palette
from matplotlib.cm import get_cmap
from bokeh.models import HoverTool
# import random, pandas as pd, numpy as np, holoviews as hv, datashader as ds, colorcet as cc
# from hv.datashader import datashade, shade, dynspread, spread, rasterize

hv.extension('bokeh', 'matplotlib', 'plotly')

TIMD_CONST = 1.002371  # TopDown isotope mass difference 55k u see OpenMS::Constants in kyowons branch

base_dir = "/tmp/results/"

In [ ]:
@dataclass
class SpecRef:
    spectrum_id: str
    tolerance: float
    massoffset: float
    chargemass: float
    isotoperangelimits: List[Tuple[int]]
    chargerangelimits: np.ndarray

@dataclass
class TargetRef:
    spectrum_id: str
    peak_index: int 
    mass: float
    charge: int
    mass_matches: np.ndarray
    intensity_matches: np.ndarray
    isotope_matches: np.ndarray

def parse_deconv_spectra_meta(spectrum: Dict[str,Any]) -> SpecRef:
  tolerance, massoffset, chargemass, *peaknotes = spectrum['DeconvMassInfo'].split(';')
  tolerance = float(tolerance.split('=')[1])
  massoffset = float(massoffset.split('=')[1])
  chargemass = float(chargemass.split('=')[1])
  peaknotes[0] = peaknotes[0].split('=')[1]  # remove 'header' before and up to '='
  peaknotes = [i.split(',') for i in peaknotes if i]  # 'if i' necessary because of trailing ','
  chargeranges, isotoperanges = list(map(list, zip(*peaknotes)))
  chargerangelimits = [tuple(map(int, i.split(':'))) for i in chargeranges]
  isotoperangelimits = [tuple(map(int, j.split(':'))) for j in isotoperanges]
  # TODO check len(spectrum) == len(chargerangelimits) == len(isotoperangelimits)
  specref = SpecRef(spectrum['id'],
                    tolerance,massoffset,chargemass,
                    chargerangelimits,isotoperangelimits)
  return specref

def delta_ppm(m1,m2):
  return (m1 - m2) / m2 * 10**6

def get_match_window(mass,tolerance):
  """
  tolerance in ppm 
  mass in m/z
  returned is window border left and window border right
  """
  tol = tolerance/2
  md = ((tol / 10**6) * mass) 
  return mass-md, mass+md  

def calc_mz(givenmass, z, iso, massoffset, chargemass):
  return (givenmass - massoffset + iso * TIMD_CONST)/z + chargemass

def calc_range(givenmass, z, iso_min, iso_max, massoffset, chargemass):
  return ((givenmass - massoffset + (iso_min-2) * TIMD_CONST)/z + chargemass, 
          (givenmass - massoffset + (iso_max+2) * TIMD_CONST)/z + chargemass )

def discharge_mz(givenmass, z, chargemass):
  return (givenmass - chargemass) * z

v_discharge_mz = np.vectorize(discharge_mz)
# TODO some peaks have 0 matches! => cannot call `vectorize` on size 0 inputs unless `otypes` is set

def get_source_peaks(range_l, range_r, source_spectrum):
  target_idx = np.where(np.logical_and(source_spectrum["m/z array"] > range_l, source_spectrum["m/z array"] < range_r))
  return source_spectrum["m/z array"][target_idx], \
          source_spectrum["intensity array"][target_idx], \
          np.ones(len(target_idx[0]))*-1

def acquire_targets_per_spectrum(deconv_spectrum: Dict[str,Any], source_spectrum: Dict[str,Any]):
  target_matches = list()
  specref = parse_deconv_spectra_meta(deconv_spectrum)
  # TODO replace for-cascade with product 
  # see (http://stephantul.github.io/python/2019/07/20/product/
  # or https://note.nkmk.me/en/python-itertools-product/)
  for i in range(0,len(deconv_spectrum["m/z array"])):
    for z in range(specref.chargerangelimits[i][0], specref.chargerangelimits[i][1]+1):
      range_l, range_r = calc_range(deconv_spectrum["m/z array"][i], z, 
                                    specref.isotoperangelimits[i][0], 
                                    specref.isotoperangelimits[i][1], 
                                    specref.massoffset, specref.chargemass)
      target_range_mz, target_range_int, target_range_iso = get_source_peaks(range_l, range_r, source_spectrum)
      if target_range_mz.size > 0:
        target_range_mass = v_discharge_mz(target_range_mz, z, specref.chargemass)
      else:
        target_range_mass = target_range_mz
      for e in range(specref.isotoperangelimits[i][0], specref.isotoperangelimits[i][1]+1):
          target_mz_iso = calc_mz(source_spectrum["m/z array"][i],z,e, specref.massoffset, specref.chargemass)
          mz_window_l, mz_window_r = get_match_window(target_mz_iso, specref.tolerance)
          np.put(target_range_iso, np.where(np.logical_and(target_range_mz > mz_window_l, target_range_mz < mz_window_r)), e)
      target_matches.append(
          TargetRef(deconv_spectrum["id"],
                    i, deconv_spectrum["m/z array"][i], z,
                    target_range_mass, target_range_int, target_range_iso)
      )
  return target_matches


In [ ]:
import os

def load_mzml():
	spec_paths = [f for f in os.listdir(base_dir) if f.endswith('.mzML')]
	comprefina = os.path.commonprefix(spec_paths)
	with mzml.read(base_dir + comprefina + "deconv.mzML") as reader:
		deconv_spectra = [spectrum for spectrum in reader]	

	with mzml.read(base_dir + comprefina + "annot.mzML") as reader:
		annot_spectra = [spectrum for spectrum in reader]

	vis_dict = dict()
	for s_a, s_o in zip(deconv_spectra, annot_spectra):
		vis_dict[s_a['id']] = acquire_targets_per_spectrum(s_a,s_o)
	return deconv_spectra, annot_spectra, vis_dict

In [ ]:
# def load_ids():
#     h5_paths = [f for f in os.listdir(base_dir) if f.endswith('.h5')]
#     if len(h5_paths) > 1:
#         print("Warning of the presence of more than one h5 file in target. Picking the first one (alphabetically).")
#     try:
#         proteoforms = pd.read_hdf(h5_paths[0], key='proteoforms')
#         prsms = pd.read_hdf(h5_paths[0], key='prsms')
#     except:
#         return None
#     return proteoforms, prsms

In [ ]:
# import nextflow 
# analysis_pipeline = nextflow.Pipeline("/opt/app/topdown_local_v2.nf", config="/opt/app/nf.conf")
# # this is a blocking process - WHY?
# apr = analysis_pipeline.run(params={
#     "raw_file": "/home/walzer/ms-tools/TopDown/TopPIC_tutorial/st_1.raw", 
#     "mods": "/home/walzer/ms-tools/TopDown/common_mods.txt", 
#     "fasta": "/home/walzer/ms-tools/TopDown/TopPIC_tutorial/TopPIC_tutorial_uniprot-st.fasta"
# })


## figure code

In [ ]:
def plot_2d_spectra(sidx, annot_spectra, deconv_spectra):
  if not sidx:
    fig_d = hv.Spikes(pd.DataFrame({},columns=['mass','intensity'])).opts(color='green', title="Deconvolved Spectrum {}".format('spectrum["id"]'))
    fig_o = hv.Spikes(pd.DataFrame({},columns=['m/z','intensity'])).opts(color='blue', title="Original Spectrum {}".format('spectrum["id"]'))
    fig_s = fig_o.opts(aspect=3, padding=0.1) + fig_d.opts(aspect=3, padding=0.1)
    fig_s.opts(aspect_weight=True, tight=False, fig_inches=300, fig_size=3).cols(1)
    return fig_s
  spectrum = deconv_spectra[sidx-2]
  ori_spectrum = annot_spectra[sidx-2]
  peak_coord = pd.DataFrame(np.concatenate([spectrum["m/z array"][np.newaxis].T, spectrum["intensity array"][np.newaxis].T], axis=1), columns = ['mass','intensity'])
  fig_d = hv.Spikes(peak_coord).opts(color='green', title="Deconvolved Spectrum {}".format(spectrum["id"]))
  peak_coord = pd.DataFrame(np.concatenate([ori_spectrum["m/z array"][np.newaxis].T, ori_spectrum["intensity array"][np.newaxis].T], axis=1), columns = ['m/z','intensity'])
  fig_o = hv.Spikes(peak_coord).opts(color='blue', title="Original Spectrum {}".format(spectrum["id"]))
  fig_s = fig_o.opts(aspect=3, padding=0.1) + fig_d.opts(aspect=3, padding=0.1)
  # fig_s = fig_o.opts(aspect=2, padding=0.10) + fig_d.opts(aspect=2, padding=0.1)
  fig_s.opts(aspect_weight=True, tight=False, fig_inches=300, fig_size=3).cols(1)
  # fig_s.opts(aspect_weight=True, tight=False).cols(1)
  return fig_s


In [ ]:
from matplotlib.cm import get_cmap
def plot_peak_map(spectra):
    # palette_inv = palette.reversed()
    # p_df = pd.DataFrame([ [s['m/z array'],s['intensity array'], s['id'], s['ms level']] for s in spectra if s['ms level']==1 ], columns = ['m/z array','intensity array', 'id', 'ms level'])
    # df = p_df.explode(['m/z array', 'intensity array'])

    import numpy as np
    import pandas as pd
    N = int(10e6)
    x_r = (0,100)
    y_r = (100,2000)
    z_r = (0,10e8)
    x = np.random.randint(x_r[0]*1000,x_r[1]*1000,size=(N, 1))
    y = np.random.randint(y_r[0]*1000,y_r[1]*1000,size=(N, 1))
    z = np.random.randint(z_r[0]*1000,z_r[1]*1000,size=(N, 1))
    z2 = np.ones((N,1)).astype(int)
    df = pd.DataFrame(np.column_stack([x,y,z,z2]), columns=['x','y','z','z2'])
    df[['x','y','z']] = df[['x','y','z']].div(1000, axis=0)

    p=hv.Points(df,['x','y'], ['z','z2'])
    palette = get_cmap('viridis')
    fig_map=rasterize(p, aggregator=ds.sum("z2"),x_range=(0,100))
    fig_map.opts(xlim=(0,100), ylim=(100,2000), cmap=palette)
    return fig_map.hist(dimension='y',weight_dimension='x_y z2',num_bins = 2000,normed=True)


In [ ]:
# Remove margins in 3D plots
# https://stackoverflow.com/a/42648316/3319796
from mpl_toolkits.mplot3d.axis3d import Axis
def _get_coord_info_new(self, renderer):
    mins, maxs, cs, deltas, tc, highs = self._get_coord_info_old(renderer)
    correction = deltas * [1.0/4 + 6.0/11,
                           1.0/4 + 6.0/11,
                           1.0/4]
    mins += correction
    maxs -= correction
    return mins, maxs, cs, deltas, tc, highs

def plot_3d_spectrum(pidx, sidx, deconv_spectra, vis_dict):
  hv.extension('matplotlib')
  if not hasattr(Axis, "_get_coord_info_old"):
      Axis._get_coord_info_old = Axis._get_coord_info  
  Axis._get_coord_info = _get_coord_info_new

  # if sidx not in deconv_spectra: 
  #   return None

  spectrum = deconv_spectra[sidx-2]
  target_matches = vis_dict[spectrum['id']]
  precursor_mass = target_matches[-1].mass
  plot_title = ' '.join([spectrum["id"].split(' ')[-1], 'precursor mass =', str(precursor_mass), 'selected peak =', str(pidx)])
  
  # molecule_mass = 12278.1611
  avrgn_masses, avrgn_ints = mass_to_dist(precursor_mass, averagine_aa, averagine_iso)

  # if pidx not in target_matches.peak_index:
    # return None

  #each peak has its matches, choice comes through pidx
  choice_peak = list(filter(lambda t: t.peak_index==pidx, target_matches))
  #for each charge there is an element in target_matches with resp. peak index that now gets formed into vis_peaks
  vis_peaks = list()
  for t in choice_peak:
    for idx, m in enumerate(t.mass_matches):
      vis_peaks.append(
          {('x', 'y', 'z'): [[t.charge,m,0],[t.charge,m,t.intensity_matches[idx]]], 
          'type': 'noise' if t.isotope_matches[idx]<0 else 'isomatch'}
      )
  #for each charge there is also the averagine model peaks for the selected peak's precursor_mass  collected in averagine_peaks
  averagine_peaks = list() 
  for c in [isoc.charge for isoc in choice_peak if isoc.mass_matches.size>0]:
    # c size of avrgn_masses zip avrgn_ints
    for m,i in zip(avrgn_masses, avrgn_ints):
      averagine_peaks.append(
          {('x', 'y', 'z'): [[c, m, i*(10e5+c*10e2)] for m,i in zip(avrgn_masses, avrgn_ints)]}
      )


  fig = hv.Path3D(vis_peaks, vdims='type').opts(color="blue", azimuth=40, elevation=20)
  # fog = hv.Scatter3D(iso_traces).opts(c='grey', s=.1, azimuth=40, elevation=20, alpha=0.1)
  fug = hv.Path3D(averagine_peaks).opts(color='grey', linewidth=.2, azimuth=40, elevation=20, alpha=0.1)
  fig_fin = (fug*fig).opts(
              ylabel="Mass",
              xlabel="Charge",
              zlabel="Intensity",
  ).opts(title=plot_title, invert_xaxis=True, fig_inches=6,)  #ax.yaxis.get_major_locator().set_params(integer=True)
  return fig_fin

## sidebar [widgets](https://panel.holoviz.org/reference/index.html#widgets)

In [ ]:
# drop_field = pn.widgets.FileInput(name="Input raw file", accept='.raw,.Raw,.RAW').servable(area="sidebar")
# drop_field.param.watch(print, 'value')

def reset(event):
    file_input.disabled = False
    progress.active = False
    print(event)

file_input = pn.widgets.FileInput(name="Input raw file").servable(area="sidebar")
progress = pn.widgets.Progress(active=False).servable(area="sidebar")

file_input.jscallback(
    args={"progress": progress},
    value="""
        progress.active = true;
        source.disabled = true;
    """
)
file_input.param.watch(reset, "value")

window = pn.widgets.StaticText(
    value="Upload a TopDown file here!", name="Announcement"
).servable(area="sidebar")

peak_selec = pn.widgets.IntSlider(name='peak_selec')
peak_df = pn.widgets.Tabulator(pd.DataFrame({},columns=['mass','intensity']))
# spectra_fig = plot_2d_spectra(None, annot_spectra, deconv_spectra)

# deconv_spectra, annot_spectra, vis_dict = load_mzml()

# def update_specs(event):
#     global peak_df
#     r = event.row
#     peak_df = pn.widgets.Tabulator(
#         pd.DataFrame(np.concatenate(
#             [deconv_spectra[r]["m/z array"][np.newaxis].T, 
#                 deconv_spectra[r]["intensity array"][np.newaxis].T],
#                     axis=1), columns = ['mass','intensity']),
#             height=500,
#     )
#     # also update vis
#     interactive_2d = plot_2d_spectra(r, annot_spectra, deconv_spectra)
#     print("update_specs")

# filecontent = pn.widgets.Tabulator(
#     pd.DataFrame({'spectrum level':[s['ms level'] for s in deconv_spectra]}),
#     height=300,
# ).servable(area="sidebar")
# filecontent.on_click=update_specs
# filecontent.selection = [0]

# spec_selec = pn.widgets.IntSlider(
#     name='spec_selec',
#     value=163, start=1, 
#     end=int(filecontent.value.size)
# ).servable(area="sidebar")

# def update_peak_selec(event):
#     global peak_df
#     peak_df = pn.widgets.Tabulator(
#         pd.DataFrame(np.concatenate(
#             [deconv_spectra[spec_selec.value]["m/z array"][np.newaxis].T, 
#                 deconv_spectra[spec_selec.value]["intensity array"][np.newaxis].T],
#                     axis=1), columns = ['mass','intensity']),
#             height=500, frozen_columns=[1],
#     )
#     peak_selec.end = int(peak_df.value.size)
#     peak_selec.start = 1
#     peak_selec.value = 1 # int(event.new)
#     print("update_peak_selec")

# spec_selec.param.watch(update_peak_selec, 'value')  

# rotation = pn.widgets.IntSlider(
#     value=40, start=0, end=180, name="3D rotation"
# ).servable(area="sidebar")

## serve panels

In [ ]:
pn.pane.Markdown("""
The selected spectrum, and from there the peaks deconvolution source(s)
""").servable()

In [ ]:
def message(sidx):
    return f"""Then details on the selected spectrum **{sidx}**."""

# def message():
#     return f"""Then details on the selected spectrum **{filecontent.selection}**."""

In [ ]:
# interactive_2d = pn.bind(plot_2d_spectra, sidx=spec_selec, deconv_spectra=deconv_spectra, annot_spectra=annot_spectra)
# interactive_text = pn.bind(message, sidx=spec_selec)
# interactive_3d = pn.bind(plot_3d_spectrum, sidx=spec_selec, pidx=peak_selec, deconv_spectra=deconv_spectra, vis_dict=vis_dict)

In [ ]:
# interactive_map = plot_peak_map(spectra=annot_spectra)
# this is way too slow to load - breaks panel
# there must be some more efficient way like:
# 1 seed the panels with empty, 
# 2 wait for file, 
# 3 process file and indicate load all over,
# 4 show 

In [ ]:

# pn.Row(pn.panel(spectra_fig, loading_indicator=True)).servable()
# pn.Row(pn.panel(interactive_2d, loading_indicator=True)).servable()
# pn.Row(interactive_text, peak_selec).servable()
# pn.Row(pn.panel(peak_df, loading_indicator=True)).servable()
# tabs = pn.Tabs(
#     ('Spectra',pn.panel(interactive_2d, loading_indicator=True))
# ).servable()
# tabs.append(
#     ('Map', pn.pane.HoloViews(interactive_map.opts(height=500, width=500, backend='bokeh')))
# ) # 
# # tabs.append(('Map', pn.panel(plot_3d_spectrum(19, 165, deconv_spectra, vis_dict), loading_indicator=True)))

# pn.Row(pn.panel(interactive_3d, loading_indicator=True)).servable()
# # pn.Row(plot_3d_spectrum(19, 163, deconv_spectra, vis_dict)).servable()

print('boop!')

## Configure the Data App

In [ ]:
pn.state.template.param.update(
    site="TopDownViz",
    title="Turn topdown mzML into deconvolved spectra visualisations",
    accent_base_color=ACCENT_BASE_COLOR,
    header_background=ACCENT_BASE_COLOR,
)

You can **serve the app** with `panel serve @filename.ipynb --autoreload` for *hot reloading* while developing. (The app is probably available at [http://localhost:5006/@filename], for correct link see the terminal log).

- For previewing the app in Jupyter lab check out the [Panel Jupyter Lab Preview](https://blog.holoviz.org/panel_0.12.0.html#JupyterLab-previews).
- For deployment options check out the [Server Deployment User Guide](https://panel.holoviz.org/user_guide/Server_Deployment.html).